미리 만들어 둔 명사 집합을 이용하여 2016-10-20 뉴스 기사를 term frequency matrix로 만든 뒤, 뉴스의 군집화를 수행합니다. Logistic Regression에 Lasso Regularization을 걸어서 각 군집의 키워드를 추출함으로써 그날 뉴스의 핫키워드를 추출해 보도록 하겠습니다. 

In [2]:
noun_dictionary_fname = './data/news_noun_dictionary.txt'
tokenized_corpus_fname = './data/news_2016-10-20_article_all_normed_nountokenized.txt'

2016-10-20일 뉴스에 대하여 CountVectorizer를 이용하여 term frequency matrix를 만든 뒤, 그날의 주요 뉴스들이 어떤 것들이 있었는지 군집화를 수행합니다. scikit learn의 KMeans는 sparse matrix로도 군집화를 수행할 수 있습니다. 

이를 위하여, 이전 수업 때 수행했던 custom_tokenizer를 이용합니다. noun_dictionary는 텍스트 파일로 저장되어 있으므로, 파일을 open으로 열 때에는 반드시 encoding='utf-8'을 신경써 주세요.

    [noun.strip() for noun in f]
    
위 구문에서 noun.strip()을 하면 줄바꿈 기호 '\n'이 사라집니다. 

In [3]:
with open(noun_dictionary_fname, encoding='utf-8') as f:
    nouns_dictionary = [noun.strip() for noun in f]
    print(len(nouns_dictionary))

def custom_tokenize(doc):    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in nouns_dictionary:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

custom_tokenize('이번 국정농단의 사태에 대하여 뉴스를 이용한 분석을 수행합니다')

45930


['이번', '국정농단', '사태', '대하여', '뉴스', '이용', '분석', '수행']

2016-10-20 뉴스에 대하여 corpus를 만듦니다. corpus의 length도 확인합니다. 당일에 30,091개의 뉴스가 있습니다. 

In [4]:
with open(tokenized_corpus_fname, encoding='utf-8') as f:
    tokenized_corpus_2016_1020 = [doc.strip() for doc in f]
len(tokenized_corpus_2016_1020)

30091

CountVectorizer를 이용하여 term frequency matrix를 만듦니다. CountVectorizer가 띄어쓰기를 기준으로 term을 구분하지만, 우리는 명사만 미리 남겨뒀으므로 띄어쓰기를 이용한 tokenizer를 해도 괜찮습니다. 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.005, max_df=0.8)
x_2016_1020 = vectorizer.fit_transform(tokenized_corpus_2016_1020)
print(x_2016_1020.shape)

(30091, 2631)


x\_2016\_1020의 각 column에 해당하는 단어를 decoding 하기 위하여 vectorizer.vocabulary\_로부터  int2vocab를 만들었습니다. 

In [8]:
int2vocab = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
int2vocab = [word for word, idx in int2vocab]
print(int2vocab[50:55])

['1년', '1명', '1시', '1시간', '1심']


만들어진 x_2016_1020을 이용하여 Spherical kmeans를 수행하겠습니다. 이를 위하여 L2로 normalization을 수행한 뒤, 클러스터의 개수를 500개로 지정하여 k-means clustering을 수행합니다. 

군집화나 토픽 모델링을 수행할 때에는 예상하는 것보다 군집/토픽의 개수를 크게 잡아주세요. 중복되는 군집이 등장하면 나중에 묶으면 됩니다. 하지만, 데이터에 노이즈가 있어서 다른 군집들이 하나의 군집으로 묶인다면 나중에 해석하기가 어려워집니다. 

특히, 여러 군집/토픽에서 두루두루 사용될 수 있는 단어들이 이러한 노이즈 역할을 합니다. 이런 단어들을 미리 걸러낼 수 있다면 훨씬 더 정교한 모델링이 될 것입니다. (즉, 군집화나 토픽모델링에서는 불필요하다 생각되는 단어들을 과감히 쳐낼수록 결과가 깔끔합니다).  그렇지 않다면, 군집화/토픽모델링을 할 때 군집/토픽의 개수를 크게 잡아두면 좀 더 좋습니다. 

어자피 k-means는 수렴을 빨리 하기 때문에 max_iter=20으로 설정하였으며, 매 iteration마다 얼마나 빠르게 수렴하는지 살펴보기 위해 verbose=1로 하였습니다. n_init=1로 설정하여 반복하지 않고 딱 한번만 군집화를 수행합니다. 

군집화를 수행한 뒤, 20개의 군집들에 대하여 각 군집에 할당된 뉴스의 개수가 몇 개인지 출력합니다. 

In [10]:
%%time

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from collections import defaultdict

x_2016_1020 = normalize(x_2016_1020, axis=1, norm='l2')
kmeans = KMeans(n_clusters=500, max_iter=20, n_init=1, verbose=1)
clusters = kmeans.fit_predict(x_2016_1020)

clusters_to_rows = defaultdict(lambda: [])
for idx, label in enumerate(clusters):
    clusters_to_rows[label].append(idx)

for i in range(20):
    print('cluster # %d has %d docs' % (i, len(clusters_to_rows[i])))

Initialization complete
Iteration  0, inertia 18954.453
Iteration  1, inertia 15373.415
Iteration  2, inertia 14879.748
Iteration  3, inertia 14658.383
Iteration  4, inertia 14524.223
Iteration  5, inertia 14447.217
Iteration  6, inertia 14403.025
Iteration  7, inertia 14373.280
Iteration  8, inertia 14349.241
Iteration  9, inertia 14333.654
Iteration 10, inertia 14323.728
Iteration 11, inertia 14315.760
Iteration 12, inertia 14311.132
Iteration 13, inertia 14308.232
Iteration 14, inertia 14306.179
Iteration 15, inertia 14304.175
Iteration 16, inertia 14301.995
Iteration 17, inertia 14299.749
Iteration 18, inertia 14297.100
Iteration 19, inertia 14294.139
cluster # 0 has 113 docs
cluster # 1 has 104 docs
cluster # 2 has 691 docs
cluster # 3 has 40 docs
cluster # 4 has 78 docs
cluster # 5 has 77 docs
cluster # 6 has 30 docs
cluster # 7 has 17 docs
cluster # 8 has 31 docs
cluster # 9 has 55 docs
cluster # 10 has 314 docs
cluster # 11 has 55 docs
cluster # 12 has 123 docs
cluster # 13 has

군집 17번은 280개의 뉴스가 묶여있습니다 (이건 군집화를 할 때마다 달라지므로, 다시 실행시키시면 다른 숫자일겁니다). 이 군집에 대하여 키워드를 추출하기 위해서는, 17번 군집에 해당하는 뉴스의 label을 1로, 다른 뉴스들을 -1로 둔 뒤, 해당 군집 17번을 구분할 수 있는 주요 단어들을 L1 regularized logistic regression을 이용하여 뽑을 수도 있습니다. 

y를 만든 뒤, 혹시 하는 마음에 x_2016_1020.shape과 같은지 확인도 해봅시다. 

In [24]:
cluster_aspect = 17
y = [1 if i in clusters_to_rows[cluster_aspect] else -1 for i in range(x_2016_1020.shape[0])]
len(y), x_2016_1020.shape

(30091, (30091, 2631))

LogisticRegression의 penalty를 'l1'으로 주고, Regularization cost를 5로 주었습니다. 키워드의 개수가 너무 많다면 C를 더 작게, 키워드의 개수가 너무 적다면 C를 좀 더 크게 조절할 수 있습니다. 

    유진 (14.231)
    진행 (13.082)
    디자이너 (11.458)
    브랜드 (10.925)
    마이데일리 (7.269)
    헤라서울패션위크 (6.670)
    동대문디자인플라자 (0.146
    
상위의 키워드를 선택하였더니, 동대문디자인플라자에서 열린 서울패션위크에 대한 이야기임을 추정할 수 있습니다. 

In [27]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=5)
logistic_l1.fit(x_2016_1020, y)

keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:30]
for word, score in keywords:
    if score == 0: break
    print('%s (%.3f)' % (int2vocab[word], score))

유진 (14.231)
진행 (13.082)
디자이너 (11.458)
브랜드 (10.925)
마이데일리 (7.269)
헤라서울패션위크 (6.670)
동대문디자인플라자 (0.146)


이번에는 관심있는 군집들에 대하여 키워드들을 뽑아내는 함수를 만들겠습니다. interested_clusters로 입력되는 클러슽들에 대하여 위처럼 각각의 클러스터에 대한 Regularized Logistic Regression을 이용하여 키워드를 추출합니다. 

Regularization cost와 키워드의 개수를 선택하기 위하여 print_keywords의 함수의 arguments에 이를 넣어둡니다. L1 Logistic Regression에서 coefficient가 0이라는 것은 classification에서 유의미한 변수가 아니라는 뜻입니다. 그렇기 때문에 아래의 구문을 넣어서 coefficient가 0보다 큰 단어들만을 선택합니다. 

    keywords = [int2vocab[word] for word, score in keywords if score > 0]

아래와 같은 결과를 얻을 수 있습니다. 단어 선택과 같은 튜닝 과정없이 명사 추출 + k-means clustering + 키워드 추출만으로도 어느 정도 그날 뉴스의 토픽들을 살펴볼 수 있습니다. 

이제 남은 일은, 좀 더 정확한 단어를 선택하여 군집화가 잘 되게 만들고, 키워드를 추출하기에 좋은 parameter를 찾는 것입니다. 

하지만 이런 접근 방법에 한 가지 단점이 있습니다. 만약 18번 군집에 대한 키워드로 '디자이너'가 선택되었지만, 이 단어는 다른 군집에서의 키워드가 될 수도 있습니다. 다른 여러개의 군집에서도 키워드로 이용될 수 있는 단어라면, 18번 군집의 키워드로 선택이 되지 않을수도 있습니다. 근본적인 원인은 18번이 아닌 다른 군집에 실제 18번 군집과 비슷한 군집이 있기 때문입니다.

In [31]:
def print_keywords(interested_clusters, x, int2vocab, clusters_to_rows, C=2, topn=30):
    
    for cluster_id in interested_clusters:
        interested_docs = clusters_to_rows[cluster_id]
        print('\ncluster # %d keywords (from %d news)' % (cluster_id, len(interested_docs)))
        y = [1 if i in interested_docs else -1 for i in range(x_2016_1020.shape[0])]
        
        logistic_l1 = LogisticRegression(penalty='l1', C=C)
        logistic_l1.fit(x, y)

        keywords = sorted(enumerate(logistic_l1.coef_[0]), key=lambda x:x[1], reverse=True)[:topn]
        keywords = [int2vocab[word] for word, score in keywords if score > 0]
        print(' > %s' % (keywords))
        
interested_clusters = [i for i, labels in clusters_to_rows.items() if 50 < len(labels) < 500]
print_keywords(interested_clusters, x_2016_1020, int2vocab, clusters_to_rows, C=2)


cluster # 0 keywords (from 113 news)
 > ['수애', '김영광', '생각', '모습', '시청률', '드라마', '로맨틱', '남자', '우리집', '라고', '구르미', '홍나리', '캐릭터', '10시', '열심히', '부담', '연하', '사랑']

cluster # 1 keywords (from 104 news)
 > ['뉴스1스타', '컬렉션', '디자이너', '서울', '헤라', '서울패션위크']

cluster # 4 keywords (from 78 news)
 > ['마이데일리', '기사', '제공', '사진', '라고', '음악', '피부', '배우', '엄마', '국내', '공개', '컬렉션', '지난', '이야기', '형사', '아빠']

cluster # 5 keywords (from 77 news)
 > ['서울경제', '공시', '20일']

cluster # 9 keywords (from 55 news)
 > ['김유정', '차태현', '기자', '박보검', '라고', '영화']

cluster # 10 keywords (from 314 news)
 > ['서울', '뉴스1', '제공', '현장', '관련', '코리아', '코스닥', '노선', '최순실', '이날', '지난', '장관', '진행', '서초구', '문화체육관광부', '19일', '공공기관', '김포공항역', '검거', '참가', '최대', '전일', '일본']

cluster # 11 keywords (from 55 news)
 > ['수상', '코리아', '2016', '기자', '서울', '시장', '개최', '기자회견', '한국일보', '20일', '규탄', '행사', '연합뉴스', '출신', '중단', '의견', '인권']

cluster # 12 keywords (from 123 news)
 > ['대통령', '최순실', '박근혜', '국민의당', '해야', '의장', '이런', '포인트', '기존', '회고록', '위기', '